In [3]:
import pickle

with open('problem_batch.pkl', "rb") as f:
    batch = pickle.load(f)

In [4]:
batch

{'cart_coords': tensor([[ 3.1212,  3.0144,  1.3837],
         [15.7608,  3.4666, -4.0248],
         [22.7381,  1.5822, -0.6263],
         ...,
         [20.7391,  1.7451, -3.7592],
         [18.1029,  1.4507, -1.0390],
         [ 7.2730,  1.2894,  3.2673]], device='cuda:0'),
 'lengths': tensor([[ 2.6139e+01,  3.7146e+00,  1.2735e+01],
         [ 1.5939e+01,  1.6019e+01,  1.1512e+01],
         [-1.0776e+03,  1.2376e+04, -5.6740e+03],
         [ 1.1420e+01,  1.1072e+01,  1.0774e+01],
         [-2.0623e+02,  2.4373e+03, -1.0956e+03],
         [ 1.1123e+01,  1.0136e+01,  8.9539e+00],
         [ 1.4447e+01,  1.4447e+01,  1.3278e+01],
         [ 1.2011e+01,  1.2223e+01,  1.5989e+01],
         [ 2.2891e+01,  2.3298e+01,  1.1510e+01],
         [ 3.8344e+01,  6.4027e+00,  1.9359e+01],
         [ 3.8584e+01,  5.2745e+00,  1.9756e+01],
         [ 3.7732e+01,  7.2513e+00,  1.8852e+01],
         [ 1.5952e+01,  1.5967e+01,  1.1312e+01],
         [ 1.2584e+01,  1.2673e+01,  1.6483e+01],
         [ 3.

In [4]:
batch.keys()

dict_keys(['cart_coords', 'lengths', 'angles', 'edge_index', 'to_jimages'])

In [6]:
print("cart_coords")
print(batch['cart_coords'])
print("lengths")
print(batch['lengths'])
print("angles")
print(batch['angles'])
print("edge_index")
print(batch['edge_index'])
print("to_jimages")
print(batch['to_jimages'])

cart_coords
tensor([[ 3.1212,  3.0144,  1.3837],
        [15.7608,  3.4666, -4.0248],
        [22.7381,  1.5822, -0.6263],
        ...,
        [20.7391,  1.7451, -3.7592],
        [18.1029,  1.4507, -1.0390],
        [ 7.2730,  1.2894,  3.2673]], device='cuda:0')
lengths
tensor([[ 2.6139e+01,  3.7146e+00,  1.2735e+01],
        [ 1.5939e+01,  1.6019e+01,  1.1512e+01],
        [-1.0776e+03,  1.2376e+04, -5.6740e+03],
        [ 1.1420e+01,  1.1072e+01,  1.0774e+01],
        [-2.0623e+02,  2.4373e+03, -1.0956e+03],
        [ 1.1123e+01,  1.0136e+01,  8.9539e+00],
        [ 1.4447e+01,  1.4447e+01,  1.3278e+01],
        [ 1.2011e+01,  1.2223e+01,  1.5989e+01],
        [ 2.2891e+01,  2.3298e+01,  1.1510e+01],
        [ 3.8344e+01,  6.4027e+00,  1.9359e+01],
        [ 3.8584e+01,  5.2745e+00,  1.9756e+01],
        [ 3.7732e+01,  7.2513e+00,  1.8852e+01],
        [ 1.5952e+01,  1.5967e+01,  1.1312e+01],
        [ 1.2584e+01,  1.2673e+01,  1.6483e+01],
        [ 3.7439e+01,  5.8612e+00,  1.880